In [118]:
import os
import pandas as pd

In [119]:
os.chdir('/Volumes/omics4tb2/SYGNAL/databases/opentargets/')

In [120]:
# %load 'sturkarslan/otparquet_serdar.py'
#!/usr/bin/env python3
"""
otparquet.py - Parquet based OpenTargets query module
"""

import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from collections import defaultdict
import argparse
import os
from tqdm.notebook import tqdm, trange
import time

def drug_info(drugdb, evidencedb, indication_map, target_map, moa_map, disease_map, warnings_map,
              drug_name, diseases, trial_phase, errors):

    # 1. grab the linked targets
    drugdf = drugdb.select("id", "name", "drugType", "isApproved", "maximumClinicalTrialPhase",
                           F.explode("linkedTargets.rows").alias("ltarget"))
    drugdf = drugdf.where(drugdf.name == drug_name)
    drug_df = drugdf.toPandas()
    linked_targets = set()
    out_item = {'molecule_name': drug_name}
    chembl_id = None
    for index, row in drug_df.iterrows():
        if row['id'] is None:  # no chembl ID => processing makes no sense
            continue
        chembl_id = row['id']
        out_item['chembl_id'] = row['id']
        out_item['drug_type'] = row['drugType']
        out_item['is_approved'] = row['isApproved']
        out_item['trial_phase'] = row['maximumClinicalTrialPhase']
        linked_targets.add(row['ltarget'])
    try:
        out_item['chembl_uri'] = 'https://www.ebi.ac.uk/chembl/compound_report_card/%s/' % out_item['chembl_id']
    except:
        error_msg = 'ERROR: could not find drug "%s" (CHEMBL_ID = %s) - skipping' % (drug_name, str(chembl_id))
        errors.append(error_msg)
        return None
    if trial_phase is not None:
        if out_item['trial_phase'] < trial_phase:
            error_msg = "WARN: drug '%s' has low trial phase - skipping" % drug_name
            errors.append(error_msg)
            return None

    # 1b. grab the evidence
    out_item['trial_urls'] = set()
    evddf = evidencedb.select('drugId', 'diseaseId', F.explode('urls').alias('url'))
    evddf = evddf.where(evddf.drugId == out_item['chembl_id'])
    evd_df = evddf.toPandas()
    for index, row in evd_df.iterrows():
        if row['url']['niceName'] == 'ClinicalTrials':
            out_item['trial_urls'].add(row['url']['url'])

    # 2. grab the indications
    out_item['indication_ids'] = []
    out_item['indication_names'] = []
    out_item['indications'] = []
    num_disease_matches = 0
    for indication in indication_map[out_item['chembl_id']]:
        disease_name = indication['efo_name']
        out_item['indications'].append({
            'id': indication['disease_id'],
            'name': indication['efo_name'],
            'max_trial_phase': indication['max_trial_phase']
        })
        out_item['indication_ids'].append(indication['disease_id'])
        out_item['indication_names'].append(indication['efo_name'])
        for d in diseases:
            try:
                disease_name.index(d)  # search for disease match
                num_disease_matches += 1
                out_item[d] = True
            except:
                out_item[d] = False
    """
    if len(diseases) > 0 and num_disease_matches == 0:
        # no matches: still include, but print warning
        error_msg = "WARN: drug '%s' has no matching diseases - skipping" % drug_name
        errors.append(error_msg)"""


    """
    # 3. grab the linked diseases
    drugdf = drugdb.select("id", "name", F.explode("linkedDiseases.rows").alias("ldisease"))
    drugdf = drugdf.where(drugdf.name == drug_name)
    drug_df = drugdf.toPandas()
    linked_diseases = set()
    for index, row in drug_df.iterrows():
        linked_diseases.add(disease_map[row['ldisease']])
    out_item['linked_diseases'] = sorted(linked_diseases)"""

    # 3. mechanisms of action
    out_item['mechanisms_of_action']  = moa_map[out_item['chembl_id']]
    # 4. linked targets
    final_linked_targets = []
    for ensembl_id in sorted(linked_targets):
        try:
            final_linked_targets.append({
                'id': ensembl_id,
                'approved_name': target_map[ensembl_id]['approved_name'],
                'approved_symbol': target_map[ensembl_id]['approved_symbol'],
                'bio_type': target_map[ensembl_id]['bio_type'],
                'target_class': target_map[ensembl_id]['target_class']
            })
        except:
            pass  # skip missing entries in targetdb
    out_item['linked_targets'] = final_linked_targets

    # 5. Warnings
    out_item['warnings'] = warnings_map[out_item['chembl_id']]
    return out_item


def make_target_map(targetdb):
    """Row(l1=Row(id=1, label='Enzyme'), l2=Row(id=643, label='Transferase'), l3=None, l4=None, l5=None, l6=None)"""
    targetdf = targetdb.select("id", "approvedName", "approvedSymbol", "bioType",
                               F.explode("proteinAnnotations.classes").alias("tclasses"))
    target_df = targetdf.toPandas()
    target_map = {}
    for index, row in target_df.iterrows():
        target_map[row['id']] = {
            'approved_name': row['approvedName'],
            'approved_symbol': row['approvedSymbol'],
            'bio_type': row['bioType'],
            'target_class': row['tclasses'].l1.label
        }
        #print(row['tclasses'])
    return target_map


def add_indication(indication_map, elem):
    """
    Row(id='CHEMBL3545136', ind=Row(disease='EFO_0000616', efoName='neoplasm', references=[Row(source='ClinicalTrials', ids=['NCT01105533'])], maxPhaseForIndication=1))"""
    try:
        indication_map[elem.ind.disease] = {
            'id': elem.ind.efoName,
            'max_trial_phase': elem.ind.maxPhaseForIndication
        }
    except:
        pass

def make_indication_map(indicationdb):
    inddf = indicationdb.select("id", F.explode("indications").alias("ind"))
    indication_map = defaultdict(list)
    for elem in inddf.collect():
        indication = {
            'disease_id': elem.ind.disease,
            'efo_name': elem.ind.efoName,
            'max_trial_phase': elem.ind.maxPhaseForIndication,
            'ref_ids': []
        }
        for ref in elem.ind.references:
            for rid in ref.ids:
                indication['ref_ids'].append(rid)
        indication_map[elem.id].append(indication)
    return indication_map

def make_moa_map(moadb):
    moadf = moadb.select("actionType", "mechanismOfAction", F.explode("chemblIds").alias("chembl_id"))
    moa_df = moadf.toPandas()
    moa_map = defaultdict(list)
    for index, row in moa_df.iterrows():
        moa_map[row['chembl_id']].append({'action_type': row['actionType'],
                                          'mechanism_of_action': row['mechanismOfAction']})
    return moa_map


def make_disease_map(diseasedb):
    diseasedf = diseasedb.select('id', 'name')
    disease_df = diseasedf.toPandas()
    disease_map = {}
    for index, row in disease_df.iterrows():
        disease_map[row['id']] = row['name']
    return disease_map


def make_warnings_map(warndb):
    warndf = warndb.select('id', 'toxicityClass', 'meddraSocCode', F.explode('chemblIDs').alias('chembl_id'))
    warn_df = warndf.toPandas()
    warnings_map = defaultdict(list)
    for index, row in warn_df.iterrows():
        if row['toxicityClass'] is not None and row['meddraSocCode'] is not None:
            warnings_map[row['chembl_id']].append({
                'toxicity_class': row['toxicityClass'],
                'meddra_soc_code': row['meddraSocCode']
            })
    return warnings_map


CSV_DELIM = ','
FIELD_DELIM = ':'

#DRUG_PATH = '/mnt/d/Projects/ISB/opentargets_queries/molecule'
#TARGET_PATH = '/mnt/d/Projects/ISB/opentargets_queries/targets'
#DISEASE_PATH = '/mnt/d/Projects/ISB/opentargets_queries/diseases'
#INDICATION_PATH = '/mnt/d/Projects/ISB/opentargets_queries/indication'
#MOA_PATH = '/mnt/d/Projects/ISB/opentargets_queries/mechanismOfAction'
#WARNINGS_PATH = '/mnt/d/Projects/ISB/opentargets_queries/drugWarnings'
#EVIDENCE_PATH = '/mnt/d/Projects/ISB/opentargets_queries/evidence/sourceId=chembl'

DBPATH="."

def drug_info_for_drugs(drugs, diseases, outdir, dbpath, trial_phase=None):
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    spark = SparkSession.builder.master('local[*]').getOrCreate()
    drugdb = spark.read.parquet(os.path.join(dbpath, 'molecule'))
    evidencedb = spark.read.parquet(os.path.join(dbpath, 'evidence/sourceId=chembl'))
    targetdb = spark.read.parquet(os.path.join(dbpath, 'targets'))
    target_map = make_target_map(targetdb)
    diseasedb = spark.read.parquet(os.path.join(dbpath, 'diseases'))
    indicationdb = spark.read.parquet(os.path.join(dbpath, 'indication'))
    #drugdb.printSchema()
    #diseasedb.printSchema()
    #indicationdb.printSchema()
    #targetdb.printSchema()
    indication_map = make_indication_map(indicationdb)
    moadb = spark.read.parquet(os.path.join(dbpath, 'mechanismOfAction'))
    moa_map = make_moa_map(moadb)
    disease_map = make_disease_map(diseasedb)
    warndb = spark.read.parquet(os.path.join(dbpath, 'drugWarnings'))
    warnings_map = make_warnings_map(warndb)

    result = []
    errors = []
    for drug_name in tqdm(drugs):
        time.sleep(0.01)
        print("Processing %s" %(drug_name))

        #chembl_id = drug_ids[drug_name]
        info = drug_info(drugdb, evidencedb, indication_map, target_map, moa_map, disease_map,
                         warnings_map, drug_name, diseases, trial_phase, errors)
        if info is not None:
            result.append(info)
            #print(info)

    # write error file
    with open(os.path.join(outdir, 'errors.txt'), 'w') as outfile:
        for error in errors:
            outfile.write(error)
            outfile.write('\n')

    with open(os.path.join(outdir, 'opentargets.csv'), 'w') as outfile:
        header = ['CHEMBL_ID', 'molecule_name', 'molecule_type',
                  'indication_ids', 'indication_names', 'max_trial_phase']
        if len(diseases) == 0:
            header.append('disease-NA')
        else:
            for d in diseases:
                header.append('disease-%s' % d)
            for d in diseases:
                header.append('max_%s_phase' % d)

        header.extend(['chembl_uri',
                       'mechanism_of_action', 'action_type', 'target_id', 'target_class',
                       'approved_name', 'literature_occ', 'trial_url',
                       'toxicity_class', 'meddra_soc_code'])

        outfile.write(CSV_DELIM.join(header))
        outfile.write('\n')
        for info in result:
            #print(info)
            row = [info['chembl_id'], info['molecule_name'], info['drug_type'],
                   FIELD_DELIM.join(info['indication_ids']),
                   '"' + FIELD_DELIM.join(info['indication_names']) + '"',
                   str(info['trial_phase'])]

            if len(diseases) == 0:
                row.append('NA')
            else:
                # add disease matches
                for d in diseases:
                    try:
                        row.append(str(info[d]).upper())
                    except:
                        row.append('FALSE')

                # add max disease phases
                for d in diseases:
                    disease_found = False
                    for ind in info['indications']:
                        if ind['name'] == d:
                            row.append(str(ind['max_trial_phase']))
                            disease_found = True
                            break
                    # fill with an empty field if no matching disease
                    if not disease_found:
                        row.append('')

            action_types = []
            moas = []
            for moa in info['mechanisms_of_action']:
                action_types.append(moa['action_type'])
                moas.append(moa['mechanism_of_action'])

            target_ids = []
            target_classes = []
            t_approved_names = []
            for t in info['linked_targets']:
                target_ids.append(t['id'])
                target_classes.append(t['target_class'])
                t_approved_names.append(t['approved_symbol'])
            row.extend([info['chembl_uri'], '"' + FIELD_DELIM.join(moas) + '"',
                        FIELD_DELIM.join(action_types),
                        FIELD_DELIM.join(target_ids),
                        FIELD_DELIM.join(sorted(set(target_classes))),
                        FIELD_DELIM.join(t_approved_names)
                        ])
            literature_occ = '' # TODO
            trial_url = '"' + FIELD_DELIM.join(list(info['trial_urls'])) + '"'
            row.append(literature_occ)
            row.append(trial_url)
            toxicity_classes = []
            meddra_soc_codes = []
            for w in info['warnings']:
                toxicity_classes.append(w['toxicity_class'])
                meddra_soc_codes.append(str(w['meddra_soc_code']))
            row.extend(['"' + FIELD_DELIM.join(toxicity_classes) + '"',
                        FIELD_DELIM.join(meddra_soc_codes)])

            outfile.write(CSV_DELIM.join(row))
            outfile.write('\n')


def drug_info_for_genes(genes, diseases, outdir, dbpath, trial_phase=None):
    # step 1: get the drug ids for the genes
    spark = SparkSession.builder.master('local[*]').getOrCreate()
    """
    targetdb = spark.read.parquet(os.path.join(dbpath, 'targets'))
    targetdb.printSchema()
    targetdf = targetdb.select("id", "approvedName", "approvedSymbol")
    targetdf.show()
    """
    print("Analyzing total of %s genes" %(len(genes))) 

    drugdb = spark.read.parquet(os.path.join(dbpath, 'molecule'))
    drugdf = drugdb.select("id", "name", "maximumClinicalTrialPhase",
                           F.explode("linkedTargets.rows").alias("ltarget"))
    drugdf = drugdf.filter(F.col('ltarget').isin(genes))
    drug_df = drugdf.toPandas()
    drug_names = set()
    for index, row in drug_df.iterrows():
        #print("Processing " + row['name'])
        drug_names.add(row['name'])
    print("Total number of drugs mapped is: %s" %(len(drug_names)))
    print()    
    print("Running drug_info_for_drugs")    
    drug_info_for_drugs(drug_names, diseases, outdir, dbpath, trial_phase)


DESCRIPTION = "otparquet - find opentargets data for drugs and disease"

if __name__ == '__main__':
    parser = argparse.ArgumentParser(formatter_class=argparse.RawDescriptionHelpFormatter,
                                     description=DESCRIPTION)
    parser.add_argument('drugs', help="list of drugs separated by new lines")
    parser.add_argument('--disease', action="append", help="disease to filter by", default=[])
    parser.add_argument('--disease_file', help="disease to filter by", default=None)
    parser.add_argument('--trial_phase', type=int, help="trial phase for drugs", default=None)
    parser.add_argument('outdir', help="output directory")
    args = parser.parse_args()

    with open(args.drugs) as infile:
        drugs = [s.strip() for s in infile]
    diseases = set(args.disease)
    if args.disease_file is not None:
        with open(args.disease_file) as infile:
            for d in infile:
                diseases.add(d.strip())

    #drugs = ['CAFFEINE', "BORTEZOMIB", "CARFILZOMIB"]
    #drugs = ["CARFILZOMIB"]
    #diseases = ['multiple myeloma']

    drug_info_for_drugs(drugs, diseases, args.outdir, DBPATH, args.trial_phase)


usage: ipykernel_launcher.py [-h] [--disease DISEASE]
                             [--disease_file DISEASE_FILE]
                             [--trial_phase TRIAL_PHASE]
                             drugs outdir
ipykernel_launcher.py: error: the following arguments are required: outdir


SystemExit: 2

In [121]:
gbmGenes = pd.read_csv("../../GBM_drug_targets/GBM_genes.csv",header=0,sep=",")
d1 = gbmGenes.genes.to_list()
d2 = d1[5001:10000]
#print(d2)

genes = d2
diseases = 'NA'
dbpath = "/Volumes/omics4tb2/SYGNAL/databases/opentargets"
outdir = "test_gene"

t1000 = drug_info_for_genes(genes=genes, diseases=diseases, dbpath=dbpath, outdir=outdir,trial_phase=None)


Analyzing total of 4999 genes


Total number of drugs mapped is: 1387

Running drug_info_for_drugs


  0%|          | 0/1387 [00:00<?, ?it/s]

Processing MP-412


Processing DOXYCYCLINE


...

Processing AMBRISENTAN


Processing AZD-7762


Processing BEMPEDOIC ACID


21/09/24 23:45:31 ERROR Executor: Exception in task 3.0 in stage 7836.0 (TID 61123)
java.io.IOException: Operation timed out
	at java.base/java.io.FileDescriptor.close0(Native Method)
	at java.base/java.io.FileDescriptor.close(FileDescriptor.java:297)
	at java.base/java.io.FileInputStream$1.close(FileInputStream.java:368)
	at java.base/java.io.FileDescriptor.closeAll(FileDescriptor.java:355)
	at java.base/java.io.FileInputStream.close(FileInputStream.java:366)
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileInputStream.close(RawLocalFileSystem.java:140)
	at java.base/java.io.BufferedInputStream.close(BufferedInputStream.java:481)
	at java.base/java.io.FilterInputStream.close(FilterInputStream.java:179)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.close(ChecksumFileSystem.java:206)
	at java.base/java.io.FilterInputStream.close(FilterInputStream.java:179)
	at org.apache.parquet.io.DelegatingSeekableInputStream.close(DelegatingSeekableInputStream.java:50)
	at 

Py4JJavaError: An error occurred while calling o176163.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 7836.0 failed 1 times, most recent failure: Lost task 2.0 in stage 7836.0 (TID 61122) (192.168.0.246 executor driver): org.apache.hadoop.fs.FSError: java.io.IOException: Device not configured
	at org.apache.hadoop.fs.RawLocalFileSystem$LocalFSFileInputStream.read(RawLocalFileSystem.java:170)
	at java.base/java.io.BufferedInputStream.fill(BufferedInputStream.java:244)
	at java.base/java.io.BufferedInputStream.read1(BufferedInputStream.java:284)
	at java.base/java.io.BufferedInputStream.read(BufferedInputStream.java:343)
	at java.base/java.io.DataInputStream.read(DataInputStream.java:148)
	at org.apache.hadoop.fs.FSInputChecker.readFully(FSInputChecker.java:460)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.readChunk(ChecksumFileSystem.java:258)
	at org.apache.hadoop.fs.FSInputChecker.readChecksumChunk(FSInputChe

In [62]:
gbmDrugs = pd.read_csv("~/Documents/Github/GbmMINER/PD-GSC/unique_GBM_drugs.csv",header=0,sep=",")
d1 = gbmDrugs.value.to_list()[1:100]
print(d1)

['CAFFEINE', 'PENTOXIFYLLINE', 'DOBUTAMINE', 'EPHEDRINE', 'EPINEPHRINE', 'NOREPINEPHRINE', 'PROPRANOLOL', 'CARVEDILOL', 'IBRUTINIB', 'TOPIRAMATE', 'GEMTUZUMAB', 'ABATACEPT', 'ABEMACICLIB', 'PALBOCICLIB', 'RIBOCICLIB', 'ROCURONIUM', 'PEGFILGRASTIM', 'REGORAFENIB', 'CETUXIMAB', 'AFATINIB', 'OSIMERTINIB', 'PANITUMUMAB', 'ERLOTINIB', 'GEFITINIB', 'LAPATINIB', 'NIMOTUZUMAB', 'DACOMITINIB', 'EPITINIB', 'NERATINIB', 'TESEVATINIB', 'VANDETANIB', 'DASATINIB', 'PERTUZUMAB', 'TRASTUZUMAB', 'ARZOXIFENE', 'FULVESTRANT', 'TOREMIFENE', 'Lenvatinib', 'PAZOPANIB', 'Nilotinib', 'INFIGRATINIB', 'DESFLURANE', 'ESZOPICLONE', 'ETOMIDATE', 'ISOFLURANE', 'MIDAZOLAM', 'PROPOFOL', 'SEVOFLURANE', 'METFORMIN', 'PERAMPANEL', 'BOSUTINIB', 'FIMEPINOSTAT', 'ROMIDEPSIN', 'BELINOSTAT', 'PANOBINOSTAT', 'VORINOSTAT', 'DEXCHLORPHENIRAMINE', 'HISTAMINE', 'ORPHENADRINE', 'CYPROHEPTADINE', 'ALDESLEUKIN', 'BASILIXIMAB', 'SARILUMAB', 'TOCILIZUMAB', 'RIBAVIRIN', 'THIOGUANINE', 'VEDOLIZUMAB', 'CILENGITIDE', 'DIAZOXIDE', 'BAFETIN

In [63]:



drugs = d1
diseases = ['multiple myeloma']
DBPATH = "/Volumes/omics4tb2/SYGNAL/databases/opentargets/"
outdir = "test"
drug_info_for_drugs(drugs,diseases,outdir,DBPATH)
#print(test.csv)

could not find drug "Lenvatinib" - skipping


could not find drug "Nilotinib" - skipping


In [34]:
drug_info_for_drugs


<function __main__.drug_info_for_drugs(drugs, diseases, args)>

In [64]:
%tb

Py4JJavaError: An error occurred while calling o1285.equalTo.
: java.lang.RuntimeException: Unsupported literal type class java.util.ArrayList [CAFFEINE]
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:90)
	at org.apache.spark.sql.catalyst.expressions.Literal$.$anonfun$create$2(literals.scala:152)
	at scala.util.Failure.getOrElse(Try.scala:222)
	at org.apache.spark.sql.catalyst.expressions.Literal$.create(literals.scala:152)
	at org.apache.spark.sql.functions$.typedLit(functions.scala:131)
	at org.apache.spark.sql.functions$.lit(functions.scala:114)
	at org.apache.spark.sql.Column.$eq$eq$eq(Column.scala:278)
	at org.apache.spark.sql.Column.equalTo(Column.scala:301)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at j